In [50]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

server = '140.114.77.15'
database = 'ChatService_EmotionTest'
username = 'sa'
password = '1Qazwsxedc'
driver= 'FreeTDS'

In [51]:
import pymssql
import codecs
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
from sklearn import metrics

conn = pymssql.connect(server=server, user=username, password=password, database=database)

In [52]:
cursor = conn.cursor()  
cursor.execute("SELECT ChatList.ChatId, ChatMessages.SenderId , ChatMessages.Content from ChatList \
                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
                WHERE ChatList.Survey = 2 AND ChatMessages.MessageType = \'text\' \
                ORDER BY ChatMessages.ChatId")  
row = cursor.fetchone() 
neg_data_sets = {}
while row:
    if row[0] in neg_data_sets:
        neg_data_sets[row[0]].append(row)
    else:
        neg_data_sets[row[0]] = [row]
    row = cursor.fetchone()

for key in list(neg_data_sets.keys()):
    user_content_array = []
    for data in neg_data_sets[key]:
        if data[1] == None or len(data[1]) > 5:
            user_content_array.append(data[2])
    neg_data_sets[key] = user_content_array

for key in list(neg_data_sets.keys()):
    if len(neg_data_sets[key]) == 0:
        del neg_data_sets[key]

In [91]:
corpus = []
satisfied_degree = []
for key in neg_data_sets:
    corpus.append(list(jieba.cut(' '.join(neg_data_sets[key]), cut_all=False)))
    satisfied_degree.append(-1)

In [125]:
corpus[2]

['那',
 ' ',
 '請問',
 'ASUS',
 '的',
 '手機',
 '有導航',
 '可',
 '使用',
 '?',
 ' ',
 'ZenFone2',
 ' ',
 '有什麼',
 '方法',
 '才',
 '有',
 '導航',
 '可',
 '使用',
 '?',
 ' ',
 '要',
 '如何',
 '使用',
 '?']

In [44]:
data_folder = './emotiondict/ANTUSD/opinion_word_utf8.csv'

word_list = []
score_list = []

f = open(data_folder, 'r')  
for row in csv.reader(f):  
    word_list.append(row[0])
    score_list.append(row[1])


In [141]:
pos_neutral_therehold = 0.05
neg_neutral_therehold = -0.05
pos_therethold = 0.2
neg_therethold = -0.2
def emotion_score_calculator(content):
    emotion_score = 0
    number = 0
    print(content)
    for word in content:
        if word in word_list:
            index = word_list.index(word)
#             print(index, word, float(score_list[index]))
            emotion_score += float(score_list[index])
            number += 1

    if number == 0:
        return emotion_score
    else:
        normalize_score = emotion_score / number
        emotion_degree = ['非常滿意', '滿意', '中立', '生氣', '非常生氣']
        if normalize_score > 0:
            if normalize_score < pos_neutral_therehold:
                return (normalize_score , emotion_degree[2])
            elif normalize_score < pos_therethold:
                return (normalize_score , emotion_degree[1])
            else:
                return (normalize_score , emotion_degree[0])
        else:
            if normalize_score > neg_neutral_therehold:
                return (normalize_score , emotion_degree[2])
            elif normalize_score > neg_therethold:
                return (normalize_score , emotion_degree[3])
            else:
                return (normalize_score , emotion_degree[4])

In [142]:
for index, data in enumerate(corpus):
    if index > 50:
        break
    print(emotion_score_calculator(data))
    print('----------------------------')

['式', '的', ' ', '是', '活', '動', '註', '冊', ' ', '經銷', '商要', '填什麼', ' ', '經銷', '商要', '填什麼', ' ', '謝謝', ' ', '經銷', '商要', '填什麼']
(0.0, '中立')
----------------------------
['如果', '也', '沒用', '呢', '？', ' ', '我', '想', '請', '問為', '什麼', '我', '的', '鬧', '鐘設', '了', '都', '不會', '叫', '嗎', ' ', '沒有', ' ', 'zefone2', ' ', 'ze551', ' ', 'ml']
(-0.17440445000000002, '生氣')
----------------------------
['那', ' ', '請問', 'ASUS', '的', '手機', '有導航', '可', '使用', '?', ' ', 'ZenFone2', ' ', '有什麼', '方法', '才', '有', '導航', '可', '使用', '?', ' ', '要', '如何', '使用', '?']
(0.24782015, '非常滿意')
----------------------------
['暗暗']
0
----------------------------
['還是', '哪', '一間', '百貨', '公司', '有', '專櫃', '的', '?', ' ', '你好', ':', '請問', '高雄市', '哪裡', '有華碩門市', '?', ' ', '可以', '體驗', '實機', '~']
(0.12758985, '滿意')
----------------------------
['沒有', '了', ' ', '如果', '維修費用', '4000', '~', '5000', '那', '就', '不修', '了', ',', '買', '一只', '新', '的', '滑算點', ' ', 'Z6', '螢幕', '破裂', '請', '問送', '哪修', ',', '費用', '多少', ' ', '喔', ',', '那', '不用', '了', '謝謝', '

(0.025223571125731002, '中立')
----------------------------
['你好', ' ', '然', '後', '拍照', '時', ',', '拍', '完上', '一張', ',', '恢', '復', '拍', '攝', '模式', '時會', '整個', '畫面', '有點', '異常', ' ', '500ML', ' ', '2G', ' ', '32G', ' ', '好', ' ', '有些', '東西', '還是', '卡卡', '的', ' ', '然後拍', '完', '後', ',', '變回', '新', '的', '拍', '攝', '模式', '時', ',', '畫面', '會', '整個', '分割', ' ', '別', '的', '版本', '記憶體', '都', '是', '超過', '1GB', '以上', '的', '記憶體', '空間', ' ', '對', '的', ' ', '過一會兒', '才', '正常', ' ', '已經', '是', '新', '的', '版本', ' ', '最新', '版本', ' ', '相機', '拍照', '時有', '問題', ',', '請惠', '於', '改善', ' ', '已經', '是', '最新', '版本', '了', ' ', '順帶', '一題', ',', ',', ',', '更新', '後', ',', '軟體', '並沒有', '很', '順暢']
(0.061728573173913046, '滿意')
----------------------------
['可惡', ',', '今天', '手機', '自動', '更新', '最新', '應用', '程式', '後', ',', '手機', '卡卡', ',', '通話', '時', '對方', '說', '授話', '雜音', '很重', ',', '到底', '是', '怎樣', ',', '爛', '公司', ',', '這是', '愛用', '國貨', '的', '代價', '嗎', '？']
(0.04344533888888888, '中立')
----------------------------
['我原', '單號', '是'

In [120]:
print(emotion_score_calculator(['一大突破', '測試']))

6 一大突破 0.2054757
0.2054757


In [121]:
print(emotion_score_calculator(corpus[101]))

17995 麻煩 -0.675
9340 其實 0.38113745
21278 新增 0.2730285
26165 應該 0.0169464
15581 問題 -0.1804542
10027 所有 0.2578135
6579 自己 0.0
3326 加入 0.0625365
1027 不了 -0.2043558
8282 沒有 -0.0921865
7162 刪除 -0.04289055
-0.2034247000000001
